In [1]:

import os

In [2]:

%pwd

'f:\\AI Projects\\End-to-End-NLP-Project-Text-Summarizer\\research'

In [3]:

os.chdir("../")

In [4]:

%pwd

'f:\\AI Projects\\End-to-End-NLP-Project-Text-Summarizer'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

In [6]:

from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name = config.tokenizer_name
        )

        return data_transformation_config

In [9]:
import os
from textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

In [12]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)

    def convert_examples_to_features(self, example_batch):

        model_inputs = self.tokenizer(
            example_batch['dialogue'],
            max_length=1024,
            truncation=True
        )

        labels = self.tokenizer(
            text_target=example_batch['summary'],
            max_length=128,
            truncation=True
        )

        model_inputs["labels"] = labels["input_ids"]

        return model_inputs

    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)

        dataset_samsum_pt = dataset_samsum.map(
            self.convert_examples_to_features,
            batched=True
        )

        dataset_samsum_pt.save_to_disk(
            os.path.join(self.config.root_dir, "samsum_dataset")
        )


In [13]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[2026-02-11 17:05:00,040: INFO: common: yaml file: config\config.yaml loaded successfully]
[2026-02-11 17:05:00,044: INFO: common: yaml file: params.yaml loaded successfully]
[2026-02-11 17:05:00,046: INFO: common: created directory at: artifacts]
[2026-02-11 17:05:00,049: INFO: common: created directory at: artifacts/data_transformation]
[2026-02-11 17:05:00,555: INFO: _client: HTTP Request: HEAD https://huggingface.co/google/pegasus-cnn_dailymail/resolve/main/config.json "HTTP/1.1 307 Temporary Redirect"]
[2026-02-11 17:05:00,578: INFO: _client: HTTP Request: HEAD https://huggingface.co/api/resolve-cache/models/google/pegasus-cnn_dailymail/40d588fdab0cc077b80d950b300bf66ad3c75b92/config.json "HTTP/1.1 200 OK"]
[2026-02-11 17:05:00,872: INFO: _client: HTTP Request: HEAD https://huggingface.co/google/pegasus-cnn_dailymail/resolve/main/tokenizer_config.json "HTTP/1.1 307 Temporary Redirect"]
[2026-02-11 17:05:00,895: INFO: _client: HTTP Request: HEAD https://huggingface.co/api/resolve-c

Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 4115.21 examples/s]
